In [29]:
import xml.etree.ElementTree as ET
import pandas as pd

# Parse XML file
tree = ET.parse('/Users/sudz4/Desktop/SUDZ4DEV/cyber-research/nmap_output/var_nmap_output_raw.xml')
root = tree.getroot()

# Initialize a list to hold all the scan data
scan_data = []

# Iterate through the hosts in the XML file
for host in root.iter('host'):
    # Get the IP address of the host
    address = host.find('address').get('addr')
    # Get the link from hostscripts if exists
    hostscript_output = ''
    hostscript = host.find('hostscript')
    if hostscript is not None:
        script = hostscript.find('script')
        if script is not None:
            hostscript_output = script.get('output')
    # Iterate through the host's ports
    for port in host.iter('port'):
        # Get the portid and protocol
        portid = port.get('portid')
        protocol = port.get('protocol')
        # Get the service name
        service = port.find('service').get('name')
        # Iterate through the scripts
        for script in port.iter('script'):
            # If the script is 'vulners', iterate through the 'table' elements
            if script.get('id') == 'vulners':
                for table in script.iter('table'):
                    # Iterate through the 'elem' elements and add the CVEs to the list
                    id_elem = table.find("elem[@key='id']")
                    cvss_elem = table.find("elem[@key='cvss']")
                    if id_elem is not None and cvss_elem is not None:
                        id_ = id_elem.text
                        cvss = cvss_elem.text
                        # Add the data for this port to the scan data
                        scan_data.append({'IP': address, 
                                          'Protocol': protocol, 
                                          'Port': portid, 
                                          'Service': service, 
                                          'CVE': id_,
                                          'CVSS': cvss})

# Create a DataFrame from the list of scan data
df = pd.DataFrame(scan_data)

# Filter the DataFrame based on CVSS >= 5.0
df = df[df['CVSS'] >= '5.0']

# Sort the DataFrame by 'CVSS' column in descending order
df = df.sort_values('CVSS', ascending=False)

# Reset the index
df = df.reset_index(drop=True)

# Define the output file path
output_file = "/Users/sudz4/Desktop/SUDZ4DEV/cyber-research/var_nmap_output_df_2_excel.xlsx"

# Save the DataFrame to an Excel file
df.to_excel(output_file, sheet_name='Nmap Vulns', index=False)

print(f'DataFrame has been written to Excel at: {output_file}')

# Print out the DataFrame
df


DataFrame has been written to Excel at: /Users/sudz4/Desktop/SUDZ4DEV/cyber-research/var_nmap_output_df_2_excel.xlsx


,IP,Protocol,Port,Service,CVE,CVSS
0,172.16.1.2,tcp,53,domain,CVE-2020-25681,8.3
1,172.16.1.2,tcp,53,domain,CVE-2020-25682,8.3
2,172.16.1.2,tcp,53,domain,SSV:96623,7.8
3,172.16.1.2,tcp,53,domain,EDB-ID:42946,7.8
4,172.16.1.2,tcp,53,domain,CVE-2017-14496,7.8
...,...,...,...,...,...,...
256,172.16.10.225,tcp,80,http,CVE-2020-1934,5.0
257,172.16.10.225,tcp,80,http,CVE-2019-17567,5.0
258,172.16.10.225,tcp,80,http,CVE-2019-0220,5.0
259,172.16.10.225,tcp,80,http,CVE-2018-17199,5.0
